In [ ]:
import pandas as pd

In [ ]:
adw_camp_df = pd.read_csv('../raw_data/adwords_campaign.csv', parse_dates=['Day'], thousands=',')

In [ ]:
adw_camp_df.info()

In [ ]:
adw_camp_df['Cost'].head(3)

In [ ]:
adw_camp_df['Cost'].apply(lambda item: item.replace('₹','').replace(',',''))\
                   .astype(float).head(3)

In [ ]:
adw_camp_df['Cost'] = adw_camp_df['Cost'].apply(lambda item: item.replace('₹','').replace(',',''))\
                                         .astype(float)

In [ ]:
adw_camp_df.info()

In [ ]:
adw_camp_df.sum()

In [ ]:
adw_conv = pd.read_csv('../raw_data/adwords_conversions.csv', parse_dates=['Day'], thousands=',')

In [ ]:
adw_conv.info()

In [ ]:
adw_conv['Conversions'] = adw_conv['Conversions'].astype('int64')

In [ ]:
adw_conv.info()

In [ ]:
adw_conv.head(3)

In [ ]:
reorder_col = ['Day', 'Campaign', 'Device', 'Conversion action', 'Conversions']

In [ ]:
adw_conv = adw_conv[reorder_col].sort_values(by=['Day', 'Campaign', 'Conversions'], ascending = [True, True, False])

In [ ]:
multi_ind = ['Day', 'Campaign', 'Device', 'Conversion action']

In [ ]:
adw_conv_ml_indx = adw_conv.set_index(multi_ind)

In [ ]:
adw_conv_ml_indx_unstack =  adw_conv_ml_indx.unstack(fill_value=0)

In [ ]:
adw_conv_ml_indx_unstack_rst_indx = adw_conv_ml_indx_unstack.reset_index()

In [ ]:
col_names = adw_conv_ml_indx_unstack_rst_indx.columns.values

In [ ]:
col_names

In [ ]:
adw_conv_ml_indx_unstack_rst_indx.columns = [''.join(col).strip().replace('Conversions', '') for col in col_names]

In [ ]:
adw_conv = adw_conv_ml_indx_unstack_rst_indx

In [ ]:
adw_camp_conv_df = adw_camp_df.merge(adw_conv,how='left',on=['Day', 'Campaign', 'Device']).fillna(0)

In [ ]:
adw_camp_conv_df.info()

In [ ]:
for item in adw_camp_conv_df.select_dtypes('float').columns.tolist()[1:]:
    adw_camp_conv_df[item] = adw_camp_conv_df[item].astype('int64')

In [ ]:
adw_camp_conv_df.info()

In [ ]:
adw_map_df = pd.read_csv('../raw_data/adwords_mapping.csv')

In [ ]:
adw_map_df.head(3)

In [ ]:
map_campaign = adw_map_df['Campaign']

In [ ]:
map_campaign.duplicated(keep='first').sum()

In [ ]:
if map_campaign.duplicated(keep='first').sum() == 0:
    print(f'There are no duplicate campaigns in the mapping file')
else:
    print(f'There are {map_campaign.duplicated(keep="first").sum()} Campaigns in the mapping file.')

In [ ]:
adw_overall = adw_camp_conv_df.merge(adw_map_df,how='left', on=['Campaign'])

In [ ]:
adw_overall['Category'].isnull().sum()

In [ ]:
adw_overall[adw_overall['Category'].isnull()]['Campaign'].drop_duplicates()

In [ ]:
if adw_overall['Category'].isnull().sum() > 0:
    print('Please add the campaign names present in the adwords_mapping_new_campaigns file to the mapping file')
    new_camps = adw_overall[adw_overall['Category'].isnull()]['Campaign'].drop_duplicates()
    new_camps.to_csv('../adwords_mapping_new_campaigns.csv', index=False)
else:
    print('Status: OK')

In [ ]:
adw_overall.head(3)

In [ ]:
adw_overall['Month'] = adw_overall['Day'].dt.strftime('%b-%y')

In [ ]:
adw_overall.head(2)

In [ ]:
del adw_overall['Conversions']

In [ ]:
for num, item in enumerate(adw_overall.columns.tolist()):
    print(num, item)

In [ ]:
col_index = [0,13,12,1,2,3,4,5,6,7,8,9,10,11]

In [ ]:
reoder_cols = [adw_overall.columns.tolist()[i] for i in col_index]

In [ ]:
adw_overall[reoder_cols].to_csv('../output/adwords_only_conversion_actions.csv', index = False)